<div align="right" style="text-align:right"><i>Peter Norvig<br>May 2015</i></div>

# When Cheryl Met Eve: A Birthday Story

The ***Cheryl's Birthday*** logic puzzle  [made the rounds](https://www.google.com/webhp?#q=cheryl%27s+birthday),
and  I wrote [code](Cheryl.ipynb) that solves it. In that notebook I said that one reason for solving the puzzle with code rather than pencil and paper is that you can do more with code.  

**[Gabe Gaster](http://www.gabegaster.com/)** proved me right when he [tweeted](https://twitter.com/gabegaster/status/593976413314777089/photo/1)  that he had used my code to generate a new list of dates that satisfies the constraints of the puzzle:

     January 15, January 4,
     July 13, July 24, July 30,
     March 13, March 24,
     May 11, May 17, May 30

In this notebook, I verify Gabe's result, and explore some new variations on the puzzle.

First, let's recap  [the original Cheryl's Birthday puzzle](https://en.wikipedia.org/wiki/Cheryl%27s_Birthday):

- Albert and Bernard became friends with Cheryl, and want to know when her birthday is.
- Cheryl wrote down a list of 10 possible dates for all to see:
    - May 15,     May 16,     May 19,
          June 17,    June 18,
          July 14,    July 16,
        August 14,  August 15,  August 17
- **Cheryl** then privately tells Albert the month and Bernard the day of her birthday.
- **Albert**: "I don't know when Cheryl's birthday is, and I know that Bernard does not know."
- **Bernard**: "At first I don't know when Cheryl's birthday is, but I know now."
- **Albert**: "Then I also know when Cheryl's birthday is."
- So when is Cheryl's birthday?

# Code for Original Cheryl's Birthday Puzzle

This is a slight modification of my [previous code](Cheryl.ipynb). The puzzle concerns these key concepts:

- **Possible dates** that might be Cheryl's birthday.
- **Knowing** which dates are still possible; knowing for sure when only one is possible.
- **Telling** Albert and Bernard specific facts about the birthday.
- **Statements** made by Albert or Bernard about their knowledge of the birthday.

I implement them as follows:
- The global variable `dates` is a set of all possible dates (each date is a string).
- `know(possible_dates)` is a function that returns `True` when there is only one possible date.
- `told(part)` is a function that returns the set of possible dates that remain after Cheryl tells a part (month or day).
- A statement is a function; *statement*`(date)`  that returns true if the statement is true given that `date` is Cheryl's birthday.
- `satisfy(possible_dates, statement,...)` returns a subset of possible_dates for which all the statements are true.

In the [previous code](Cheryl.ipynb) I treated `dates` as a constant, but in this version the whole point is exploring different sets of possible dates. The easiest way to refactor the code was to make `dates` a global variable, and provide the function `update_dates` to set the value of the global variable. (It would be cleaner to package the dates into a non-global object, but it would be a big change to the  code to inject this all the way down to the function `told`, where it is needed.)

In [1]:
# Albert and Bernard just became friends with Cheryl, and they want to know when her birthday is. 
# Cheryl gave them a list of 10 possible dates:

dates = {'May 15',    'May 16',    'May 19',
        'June 17',   'June 18',
        'July 14',   'July 16',
      'August 14', 'August 15', 'August 17'}

def month(date): return date.split()[0]
def day(date):   return date.split()[1]

# Cheryl then tells Albert and Bernard separately 
# the month and the day of the birthday respectively.

BeliefState = set

def told(part: str) -> BeliefState:
    """Cheryl told a part of her birthdate to someone; return a belief state of possible dates."""
    return {date for date in dates if part in date}

def know(beliefs: BeliefState) -> bool:
    """A person `knows` the answer if their belief state has only one possibility."""
    return len(beliefs) == 1

def satisfy(some_dates, *statements) -> BeliefState:
    """Return the subset of dates that satisfy all the statements."""
    return {date for date in some_dates
            if all(statement(date) for statement in statements)}

# Albert and Bernard make three statements:

def albert1(date) -> bool:
    """Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."""
    albert_beliefs = told(month(date))
    return not know(albert_beliefs) and not satisfy(albert_beliefs, bernard_knows)

def bernard_knows(date) -> bool: return know(told(day(date))) 

def bernard1(date) -> bool:
    """Bernard: At first I don't know when Cheryl's birthday is, but I know now."""
    at_first_beliefs = told(day(date))
    after_beliefs   = satisfy(at_first_beliefs, albert1)
    return not know(at_first_beliefs) and know(after_beliefs)

def albert2(date) -> bool:
    """Albert: Then I also know when Cheryl's birthday is."""
    then = satisfy(told(month(date)), bernard1)
    return know(then)
    
# So when is Cheryl's birthday?

def cheryls_birthday(possible_dates) -> BeliefState:
    """Return a subset of the dates for which all three statements are true."""
    return satisfy(update_dates(possible_dates), albert1, bernard1, albert2)

def update_dates(possible_dates) -> BeliefState:
    """Set the value of the global `dates` to `possible_dates`."""
    global dates
    dates = possible_dates
    return dates

In [2]:
# Some tests

assert month('May 19') == 'May'
assert day('May 19') == '19'
assert albert1('May 19') == False
assert albert1('July 14') == True
assert know(told('16')) == False
assert know(told('19')) == True
assert know(told('May')) == False

Below we trace through how this works.

First Albert says that he doesn't know, and that Bernard doesn't either. So the possible remaining dates are:

In [3]:
satisfy(dates, albert1)

{'August 14', 'August 15', 'August 17', 'July 14', 'July 16'}

Bernard says he initially didn't know, but now he does. He knows, but we the puzzle-solvers don't. The remaining possible dates for us are:

In [4]:
satisfy(dates, albert1, bernard1)

{'August 15', 'August 17', 'July 16'}

Now Albert knows, and so do we:

In [5]:
satisfy(dates, albert1, bernard1, albert2)

{'July 16'}

In [6]:
cheryls_birthday(dates)

{'July 16'}

# Verifying Gabe's Version

Gabe tweeted these ten dates:

In [7]:
gabe_dates = [
  'January 15', 'January 4',
  'July 13',    'July 24',   'July 30',
  'March 13',   'March 24',
  'May 11',     'May 17',    'May 30']

We can verify that they do indeed make the puzzle work:

In [8]:
cheryls_birthday(gabe_dates)

{'July 30'}

# Creating Our Own Versions

If Gabe can do it, we can do it!  Our strategy will be to repeatedly pick a random sample of dates, and check if they solve the puzzle. We'll limit ourselves to a subset of dates (not all 366) to make it more likely that a random selection will have multiple dates with the same month and day (otherwise Albert and/or Bernard would know right away):

In [9]:
many_dates = {mo + ' ' + d1 + d2
              for mo in {'April', 'August', 'July', 'June', 'March', 'May'}
              for d1 in '12'
              for d2 in '3456789'}

Now we need to cycle through random samples of these possible dates until we hit one that works.  I anticipate wanting to solve other puzzles besides the original `cheryls_birthday`, so I'll define  the function `pick_dates` to take a parameter, `puzzle`:

In [10]:
import random

def pick_dates(puzzle, k=10) -> BeliefState:
    """Pick a set of `k` dates for which the `puzzle` has a unique solution."""
    while True:
        random_dates = random.sample(many_dates, k)
        solutions = puzzle(random_dates)
        if know(solutions):
            return set(random_dates)

In [11]:
pick_dates(cheryls_birthday)

{'April 18',
 'August 19',
 'August 28',
 'July 15',
 'June 14',
 'June 19',
 'June 28',
 'May 14',
 'May 19',
 'May 24'}

In [12]:
pick_dates(cheryls_birthday, k=6)

{'July 16', 'July 18', 'July 29', 'March 16', 'March 18', 'May 29'}

In [13]:
pick_dates(cheryls_birthday, k=12)

{'April 23',
 'April 24',
 'April 29',
 'August 19',
 'August 24',
 'August 29',
 'June 13',
 'June 23',
 'March 14',
 'March 15',
 'May 13',
 'May 27'}

Great! We can make a new puzzle, just like Gabe.  But how often do we get a unique solution to the puzzle (that is, the puzzle returns a set of size 1)?  How often do we get a solution where Albert and Bernard know, but we the puzzle solver don't (that is, a belief set of size greater than 1)?  How often is there no solution (size 0)? Let's make a Counter of the number of times each length-of-solution occurs:

In [14]:
from collections import Counter

def solution_lengths(puzzle, N=10000, k=10, many_dates=many_dates):
    """Try N random samples of k dates and count how often each possible 
    length-of-puzzle-solution appears."""
    return Counter(len(puzzle(random.sample(many_dates, k)))
                   for _ in range(N))

In [15]:
solution_lengths(cheryls_birthday)

Counter({0: 9414, 2: 380, 1: 206})

This says that about 2% of the time we get a unique solution (a set of length 1). More often than that we get an ambiguous solution (with 2 or more possible birth dates), but about 95% of the time the sample of dates has no solution (a set of length 0).

What happens if Cheryl changes the number of possible dates?

In [16]:
solution_lengths(cheryls_birthday, k=6)

Counter({0: 9987, 1: 3, 2: 10})

In [17]:
solution_lengths(cheryls_birthday, k=12)

Counter({0: 8821, 2: 632, 1: 537, 3: 10})

It is  hard (but not impossible) to find a set of 6 dates that work for the puzzle, and much easier to find a solution with 12 dates.

# A New Puzzle: All About Eve

Now let's see if we can create a more complicated puzzle. We'll introduce a new character, Eve, and keep the same puzzle as before,e xcept that after Albert's second statement, Eve makes this statement:

- **Eve**: "Hi, Everybody. My name is Eve and I'm an evesdropper. It's what I do! I peeked and saw the first letter of the month and the first digit of the day. When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard I do.  And it's a good thing I peeked, because otherwise I couldn't have
figured it out."

We can easily code this up:

In [18]:
def cheryls_birthday_with_eve(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return satisfy(update_dates(dates), albert1, bernard1, albert2, eve1)

def eve1(date):
    """Eve: I peeked and saw the first letter of the month and the first digit of the day. 
    When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard 
    I do. And it's a good thing I peeked, because otherwise I couldn't have figured it out."""
    at_first = told(day(date)[0]) & told(month(date)[0])
    return (not know(at_first) and
                know(satisfy(at_first,  albert1, bernard1, albert2)) and
            not know(satisfy(dates, albert1, bernard1, albert2)))

*Note*: I admit I "cheated" a bit here.  Remember that the function `told`  tests for `(part in date)`.  For that to work for Eve, we have to make sure that the first letter is distinct from any other character in the date (it is&mdash;because only the first letter is uppercase) and that the first digit is distinct from any other character (it is&mdash;because in `many_dates` I carefully made sure that the first digit is always 1 or 2, and the second digit is never 1 or 2). 

I have no idea if it is possible to find a set of dates that works for this puzzle. But I can try:

In [19]:
pick_dates(puzzle=cheryls_birthday_with_eve)

{'April 26',
 'August 25',
 'June 15',
 'June 19',
 'June 23',
 'June 29',
 'March 13',
 'March 23',
 'May 13',
 'May 19'}

That was easy.  How often is a random sample of dates a solution to this puzzle?

In [20]:
solution_lengths(cheryls_birthday_with_eve)

Counter({0: 9708, 1: 138, 2: 154})

A solution (a set of length 1) occurs a bit less often than with the original puzzle.

# An Even More Complex Puzzle

Let's make the puzzle even more complicated by making Albert wait one more time before he finally knows:

- Albert and Bernard just became friends with Cheryl, and they want to know when her birtxhday is. Cheryl wrote down a list of 10 possible dates for all to see.
- **Cheryl** then writes down the month and shows it just to Albert, and also writes down the day and shows it just to Bernard.
- **Albert**: I don't know when Cheryl's birthday is, but I know that Bernard does not know either. 
- **Bernard**: At first I didn't know when Cheryl's birthday is, but I know now.
- **Albert**: I still don't know.
- **Eve**: Hi, Everybody. My name is Eve and I'm an evesdropper. It's what I do! I peeked and saw the first letter of the month and the first digit of the day. When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard I do.  And it's a good thing I peeked, because otherwise I couldn't have
figured it out.
- **Albert**: OK, now I know.
- So when is Cheryl's birthday?

Albert's second statement is different; he has a new third statement; and Eve's statement uses the same words, but it now implicitly refers to a different statement by Albert. We'll use the names `albert2c`,  `albert3c`, and `eve1c` (`c` for "complex") to represent the new statements:

In [21]:
def cheryls_birthday_complex(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return satisfy(update_dates(dates), albert1, bernard1, albert2c, eve1c, albert3c)

def albert2c(date):
    "Albert: I still don't know."
    return not know(satisfy(told(month(date)), bernard1))

def eve1c(date):
    """Eve: I peeked and saw the first letter of the month and the first digit of the day. 
    When I peeked, I didn't know Cheryl's birthday, but after listening to Albert and Bernard 
    I do. And it's a good thing I peeked, because otherwise I couldn't have figured it out."""
    at_first = told(day(date)[0]) & told(month(date)[0])
    return (not know(at_first)
            and know(satisfy(at_first, albert1, bernard1, albert2c)) and
            not know(satisfy(dates, albert1, bernard1, albert2c)))

def albert3c(date):
    "Albert: OK, now I know."
    return know(satisfy(told(month(date)), bernard1, eve1c))

Again, I don't know if it is possible to find dates that works with this story, but I can try:

In [22]:
pick_dates(puzzle=cheryls_birthday_complex)

{'April 16',
 'April 28',
 'August 13',
 'August 18',
 'August 23',
 'August 26',
 'August 27',
 'July 16',
 'July 27',
 'June 29'}

It worked!  Were we just lucky, or are there many sets of dates that work?

In [23]:
solution_lengths(cheryls_birthday_complex)

Counter({0: 9207, 1: 790, 2: 3})

Interesting. It was actually easier to find dates that work for this story than for any of the other stories.

## Analyzing a Solution to the Complex Puzzle

Now we will go through a solution step-by-step.  We'll use these dates:

In [24]:
complex_dates = {
  'April 28',
  'July 27',
  'June 19',
  'June 16',
  'July 15',
  'April 15',
  'June 29',
  'July 16',
  'May 24',
  'May 27'}

Let's find the solution:

In [25]:
cheryls_birthday_complex(complex_dates)

{'July 27'}

Now the first step is that Albert was told "July":

In [26]:
told('July')

{'July 15', 'July 16', 'July 27'}

And no matter which of these three dates is the actual birthday, Albert knows that Bernard would not know the birthday, because each of the days (15, 16, 27) appears twice in the list of possible dates.

In [27]:
not know(told('15')) and not know(told('16')) and not know(told('27'))

True

Meanwhile, Bernard is told the day:

In [28]:
told('27')

{'July 27', 'May 27'}

There are two dates with a 27, so Bernard did not know then. But only one of these dates is still consistent after hearing Albert's statement:

In [29]:
satisfy(told('27'), albert1)

{'July 27'}

So after Albert's statement, Bernard knows. Poor Albert still doesn't know (after being told `'July'` and hearing Bernard's statement):

In [30]:
satisfy(told('July'), bernard1)

{'July 15', 'July 16', 'July 27'}

Then along comes Eve. She evesdrops the "J" and the "2":

In [31]:
told('J') & told('2')

{'July 27', 'June 29'}

Two dates, so Eve doesn't know after evesdropping. But only one of the dates works after hearing the three statements made by Albert and Bernard:

In [32]:
satisfy(told('J') & told('2'), albert1, bernard1, albert2c)

{'July 27'}

But Eve wouldn't have known if she hadn;t been told anything:

In [33]:
satisfy(dates, albert1, bernard1, albert2c)

{'July 15', 'July 16', 'July 27'}

What about Albert?  After hearing Eve's statement he finally knows:

In [34]:
satisfy(told('July'), eve1c)

{'July 27'}

# What Next?

There are many other directions you could take this:

- Could you create a puzzle that goes one or two rounds more before everyone knows?
- Could you add new characters: Faith, and then George, and maybe even a new Hope?
- Would it be more interesting with a different number of possible dates (not 10)?
- Should we include the year or the day of the week, as well as the month and day?
- Perhaps a puzzle that starts with [Richard Smullyan](http://en.wikipedia.org/wiki/Raymond_Smullyan) announcing that one of the characters is a liar.
- Or you could make a puzzle harder than [the hardest logic puzzle ever](https://en.wikipedia.org/wiki/The_Hardest_Logic_Puzzle_Ever).
- Try the "black and white hats" [Riddler Express](https://fivethirtyeight.com/features/can-you-solve-these-colorful-puzzles/) stumper.
- It's up to you ...